In [156]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

# data = pd.read_csv("../input/sample.csv")
data = pd.read_csv('data/twcs.csv')

In [157]:
data.shape

(1904290, 7)

In [158]:
initial_inbound_tweets = data[data['inbound'] & pd.isnull(data['in_response_to_tweet_id'])]

In [159]:
# Pick only inbound tweets that aren't in reply to anything...
first_inbound = data[pd.isnull(data.in_response_to_tweet_id) & data.inbound]

# Merge in all tweets in response
inbounds_and_outbounds = pd.merge(first_inbound, data, left_on='tweet_id', 
                                  right_on='in_response_to_tweet_id')
print(inbounds_and_outbounds.columns)
# Filter out cases where reply tweet isn't from company
inbounds_and_outbounds = inbounds_and_outbounds[inbounds_and_outbounds.inbound_y ^ True]

Index(['tweet_id_x', 'author_id_x', 'inbound_x', 'created_at_x', 'text_x',
       'response_tweet_id_x', 'in_response_to_tweet_id_x', 'tweet_id_y',
       'author_id_y', 'inbound_y', 'created_at_y', 'text_y',
       'response_tweet_id_y', 'in_response_to_tweet_id_y'],
      dtype='object')


In [160]:
# inbounds_and_outbounds.reset_index()
# Count total volume per company
total_company_tweets = inbounds_and_outbounds.groupby("author_id_y").count()["tweet_id_x"]

# Figure out the top 5 companies by volume.
total_company_tweets.sort_values(ascending=False)[0:5]

author_id_y
AppleSupport    56197
AmazonHelp      55389
Uber_Support    25602
Delta           20492
SpotifyCares    18152
Name: tweet_id_x, dtype: int64

In [161]:
apple_support = inbounds_and_outbounds[inbounds_and_outbounds['author_id_y'] == 'AmazonHelp']

In [162]:
def format_tweet(tweet):
    # Remove Twitter handles & hashtags
    entity_prefixes = ['@','#', 'http']
    tweet_without_entities = " ".join([word for word in tweet.split() if not word.startswith(tuple(entity_prefixes))])
    
    # Remove non-alpha characters
    return ''.join([char for char in tweet_without_entities if not char.isdigit()])

formatted_tweets = apple_support['text_x'].apply(format_tweet)

# Quick comparison pre-formatting & post-formatting.
print("{}".format(apple_support['text_x'].iloc[0]))
print("->")
print("{}".format(formatted_tweets.iloc[0]))

Dear @AmazonHelp  It feels  really devastating that once you promise something then you break it. I didn't get my cashback yet. See it. https://t.co/6mQg6QpQdc
->
Dear It feels really devastating that once you promise something then you break it. I didn't get my cashback yet. See it.


In [170]:
import re
no_features = 10000

tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tfidf = tfidf_vectorizer.fit_transform(formatted_tweets)
tfidf_feature_names = tfidf_vectorizer.get_feature_names()

# LDA can only use raw term counts for LDA because it is a probabilistic graphical model
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
tf = tf_vectorizer.fit_transform(formatted_tweets)
tf_feature_names = tf_vectorizer.get_feature_names()

In [171]:
no_topics = 10

# Run NMF
nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

# Run LDA
lda = LatentDirichletAllocation(n_components=no_topics, max_iter=5, learning_method='online', learning_offset=50.,random_state=0).fit(tf)

In [172]:
def display_topics(H, W, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("Topic {}".format(topic_idx + 1))
        print("Feature Names: {}".format(", ".join([feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]])))
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:no_top_documents]
        for i, doc_index in enumerate(top_doc_indices):
            print("Example #{}: {}".format(i + 1, documents.iloc[doc_index]))
        print()

In [173]:
nmf_W = nmf.transform(tfidf)
nmf_H = nmf.components_

# Run LDA
lda_W = lda.transform(tf)
lda_H = lda.components_

no_top_words = 5
no_top_documents = 5

In [174]:
display_topics(nmf_H, nmf_W, tfidf_feature_names, apple_support['text_x'], no_top_words, no_top_documents)

Topic 1
Feature Names: order, placed, id, cancelled, cancel
Example #1: @AmazonHelp Where's my order?!!!!!
Example #2: Why can't I order toasted ravioli @84042
Example #3: #AmazonPrime #Amazonin @84039 #order #delivered but not #received #deliveryissue #nofollowup #complaintmade Order#407-1918105-8965111
Example #4: @84039 my order no is

404-6253992-4618766
Example #5: @84039  order Quary #405-7877725-5465914

Topic 2
Feature Names: amazon, india, pay, account, balance
Example #1: Amazon Studentの2000円クーポン
使えないんだけどなんで？
貰えるのを前提に登録したのに…
ちゃんとAmazon販売配送商品なのに…
かれこれ2時間格闘してる泣きそう
Example #2: Amazon primeはいいぞ。 https://t.co/LpWUpytkz0
Example #3: 昨日からスピッツしばり。

Amazon Music本当に便利だからこのためだけにprime会員になってもいいレベルだと思うの…

#AmazonPrime https://t.co/jBwAmbAdjb
Example #4: Amazon bar行きたかった人生であった。
Example #5: amazon bar良さげやな

Topic 3
Feature Names: prime, membership, days, time, member
Example #1: @84042 How is this Prime? https://t.co/zPzOXmJiq2
Example #2: Prime readingとkindle unlimitedの区別ってどうされるの？両会員だけどアンリミ

In [175]:
display_topics(lda_H, lda_W, tf_feature_names, apple_support['text_x'], no_top_words, no_top_documents)

Topic 1
Feature Names: account, time, amazon, money, deliver
Example #1: @amazonhelp do you make most of your money by purposely making it impossible for customers to receive help on incorrect orders?
Example #2: Has @85465 increased the rate from 499pa to 999pa? Is it even for existing customers? @84039
Example #3: Too many times now, my prime orders have been delayed, @84039! Your reliability is becoming questionable.
#AmazonPrime
Example #4: Always used to have really reliable prime deliveries from @84028 but the last two have been a mess 🤨
Example #5: .@84042 sooo if some1 hacked my acct, changed e-mail address, purchased things on this acct, what should I do? Your cust svc no help at all

Topic 2
Feature Names: just, delivery, time, package, come
Example #1: @84042 The language barrier with your CSR’s SUCKS! I have called 4 times talked to 4 different people who can’t understand shit I say!
Example #2: @84398 misses delivery window once again to continue their streak of not a sing

In [176]:
import pyLDAvis
import pyLDAvis.sklearn
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning) 
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tfidf, tfidf_vectorizer)

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
6      14.367488        1       1 -0.205527  0.090674
9      13.914598        1       2 -0.227403  0.093283
8      10.787841        1       3 -0.082827 -0.038472
3       9.893302        1       4  0.291479  0.085273
2       9.427178        1       5 -0.145069 -0.090526
4       8.993833        1       6  0.004762  0.222795
0       8.562929        1       7  0.009557  0.083190
1       8.480715        1       8 -0.019141 -0.222793
7       8.148025        1       9  0.092269 -0.292873
5       7.424091        1      10  0.281899  0.069448, topic_info=     Category         Freq       Term        Total  loglift  logprob
term                                                                
5902  Default  2576.000000      order  2576.000000  30.0000  30.0000
6556  Default  1758.000000      prime  1758.000000  29.0000  29.0000
6629  Default   951.000000    product   951.000000  28.0000  28.0000
7729  Default  1142.000000    service  1142.000000  27.0000  27.0000
2107  Default   976.000000        day   976.000000  26.0000  26.0000
4671  Default   783.000000       just   783.000000  25.0000  25.0000
2256  Default  2131.000000   delivery  2131.000000  24.0000  24.0000
2013  Default   908.000000   customer   908.000000  23.0000  23.0000
60    Default   649.000000    account   649.000000  22.0000  22.0000
8805  Default   875.000000       time   875.000000  21.0000  21.0000
3911  Default   466.000000       guys   466.000000  20.0000  20.0000
2251  Default  1201.000000  delivered  1201.000000  19.0000  19.0000
450   Default   451.000000        app   451.000000  18.0000  18.0000
5904  Default   945.000000    ordered   945.000000  17.0000  17.0000
3825  Default   612.000000        got   612.000000  16.0000  16.0000
7114  Default   514.000000     refund   514.000000  15.0000  15.0000
9276  Default   454.000000         ve   454.000000  14.0000  14.0000
294   Default  2646.000000     amazon  2646.000000  13.0000  13.0000
5474  Default   397.000000      money   397.000000  12.0000  12.0000
2246  Default   551.000000    deliver   551.000000  11.0000  11.0000
6827  Default   543.000000        que   543.000000  10.0000  10.0000
4953  Default   450.000000       like   450.000000   9.0000   9.0000
2109  Default   810.000000       days   810.000000   8.0000   8.0000
4595  Default   423.000000       item   423.000000   7.0000   7.0000
7807  Default   438.000000   shipping   438.000000   6.0000   6.0000
5989  Default   784.000000    package   784.000000   5.0000   5.0000
4083  Default   895.000000       help   895.000000   4.0000   4.0000
6996  Default   576.000000   received   576.000000   3.0000   3.0000
337   Default   726.000000        amp   726.000000   2.0000   2.0000
7346  Default   403.000000     return   403.000000   1.0000   1.0000
...       ...          ...        ...          ...      ...      ...
6075  Topic10   178.234582        pas   178.514251   2.5989  -4.0785
3059  Topic10   151.508288         et   151.787958   2.5986  -4.2409
7939  Topic10   147.682501       site   147.962223   2.5985  -4.2665
210   Topic10   139.428318         ai   139.707991   2.5984  -4.3240
6453  Topic10   133.186324       pour   133.465993   2.5983  -4.3698
1569  Topic10   128.905766      colis   129.185435   2.5983  -4.4025
9595  Topic10   129.727608    website   130.009373   2.5983  -4.3961
9490  Topic10   124.243618       vous   124.523288   2.5982  -4.4393
5468  Topic10   121.311676        mon   121.591356   2.5981  -4.4632
6233  Topic10   119.748113     pickup   120.027828   2.5981  -4.4762
6229  Topic10   110.057371       pick   110.337091   2.5979  -4.5606
9102  Topic10   101.159535        une   101.439204   2.5977  -4.6449
1054  Topic10    83.113000    bonjour    83.392670   2.5971  -4.8414
4894  Topic10    82.557376        les    82.837052   2.5971  -4.8481
4991  Topic10    81.267622  livraison    81.547291   2.5970  -4.86

In [177]:
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(nmf, tf, tf_vectorizer)

/Users/charleschanlee/anaconda/lib/python3.6/site-packages/pyLDAvis/sklearn.py:26: RuntimeWarning: invalid value encountered in true_divide
  return dists / dists.sum(axis=1)[:, None]
/Users/charleschanlee/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:223: RuntimeWarning: divide by zero encountered in log
  kernel = (topic_given_term * np.log((topic_given_term.T / topic_proportion).T))
/Users/charleschanlee/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:240: RuntimeWarning: divide by zero encountered in log
  log_lift = np.log(topic_term_dists / term_proportion)
/Users/charleschanlee/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:241: RuntimeWarning: divide by zero encountered in log
  log_ttd = np.log(topic_term_dists)
/Users/charleschanlee/anaconda/lib/python3.6/site-packages/pyLDAvis/_prepare.py:208: RuntimeWarning: invalid value encountered in multiply
  relevance = lambda_ * log_ttd + (1 - lambda_) * log_lift
/Users/charleschanlee/anaconda/lib/pyt

PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
2      17.298778        1       1 -0.011995  0.226514
7      14.965467        1       2 -0.064549 -0.081849
1      14.222154        1       3 -0.238310  0.093901
0       9.812732        1       4 -0.012557 -0.155415
6       9.317579        1       5  0.142208 -0.034261
3       8.674284        1       6 -0.188593 -0.050014
5       7.884936        1       7 -0.206480 -0.058385
9       6.787743        1       8  0.177550  0.013985
8       5.623650        1       9  0.177015  0.236447
4       5.412677        1      10  0.225710 -0.190922, topic_info=     Category          Freq        Term         Total  loglift  logprob
term                                                                   
294   Default  21090.000000      amazon  21090.000000  30.0000  30.0000
5902  Default  11458.000000       order  11458.000000  29.0000  29.0000
6556  Default  13957.000000       prime  13957.000000  28.0000  28.0000
2251  Default   7578.000000   delivered   7578.000000  27.0000  27.0000
4083  Default   8131.000000        help   8131.000000  26.0000  26.0000
2256  Default   8559.000000    delivery   8559.000000  25.0000  25.0000
5989  Default   6234.000000     package   6234.000000  24.0000  24.0000
2107  Default   5921.000000         day   5921.000000  23.0000  23.0000
7729  Default   5937.000000     service   5937.000000  22.0000  22.0000
2013  Default   4944.000000    customer   4944.000000  21.0000  21.0000
7807  Default   3072.000000    shipping   3072.000000  20.0000  20.0000
6629  Default   4699.000000     product   4699.000000  19.0000  19.0000
5612  Default   3202.000000        need   3202.000000  18.0000  18.0000
5309  Default   2771.000000  membership   2771.000000  17.0000  17.0000
2109  Default   4806.000000        days   4806.000000  16.0000  16.0000
7114  Default   3026.000000      refund   3026.000000  15.0000  15.0000
60    Default   3761.000000     account   3761.000000  14.0000  14.0000
8843  Default   3660.000000       today   3660.000000  13.0000  13.0000
6996  Default   3641.000000    received   3641.000000  12.0000  12.0000
9754  Default   2102.000000       worst   2102.000000  11.0000  11.0000
7563  Default   2331.000000        says   2331.000000  10.0000  10.0000
5904  Default   3690.000000     ordered   3690.000000   9.0000   9.0000
6115  Default   3265.000000         pay   3265.000000   8.0000   8.0000
4286  Default   1548.000000          id   1548.000000   7.0000   7.0000
6279  Default   1218.000000      placed   1218.000000   6.0000   6.0000
7346  Default   1790.000000      return   1790.000000   5.0000   5.0000
5307  Default   1622.000000      member   1622.000000   4.0000   4.0000
1293  Default   1504.000000        care   1504.000000   3.0000   3.0000
6108  Default   1552.000000    pathetic   1552.000000   2.0000   2.0000
4423  Default   2008.000000       india   2008.000000   1.0000   1.0000
...       ...           ...         ...           ...      ...      ...
8009  Topic10    168.524735         sms    334.118781   2.2320  -4.9909
8252  Topic10    483.470811      status   1155.303585   2.0453  -3.9370
5626  Topic10     79.233030   neighbour    142.807226   2.3273  -5.7456
8470  Topic10    479.674251    supposed   1320.209776   1.9040  -3.9449
3964  Topic10    103.255059      handed    206.721570   2.2223  -5.4808
8900  Topic10    394.507280    tracking   1142.760308   1.8529  -4.1404
9832  Topic10    440.211495   yesterday   1353.236875   1.7934  -4.0307
8843  Topic10    916.725453       today   3660.669969   1.5318  -3.2972
5521  Topic10    132.178669         msg    313.984881   2.0512  -5.2338
4901  Topic10     20.742938   letterbox     28.529278   2.5977  -7.0858
4012  Topic10    293.021762        hasn   1133.258423   1.5638  -4.4377
7507  Topic10    242.120669        said    946.350750   1.5532  -4.6286
106   Topic10    141.902470    actually    494.144147   1.6687  -5.16